# Analyse et préparation des données — Beyond GDP

Ce notebook a pour objectif de charger, explorer, nettoyer et structurer la base de données utilisée dans le dashboard interactif Beyond GDP.
Il détaille l’ensemble du processus, depuis le fichier brut issu du World Development Indicators (WDI) jusqu’à la production du fichier final prêt à être utilisé dans Streamlit.

### 🎯 Objectifs du notebook

- Explorer la structure du dataset WDI (taille, colonnes, pays, indicateurs).

- Vérifier la qualité : valeurs manquantes, doublons, cohérence temporelle.

- Sélectionner les indicateurs pertinents par thématique :
Économie, Santé, Éducation, Environnement, Inégalités, Société.

- Transformer le format du fichier (wide → long) pour faciliter l’analyse.

- Nettoyer et standardiser : typage, renommage, filtrage des années (1980–2023).

- Générer un fichier propre, optimisé pour la visualisation interactive.

### 🧩 Structure des données

Le dataset brut WDI contient :

- plus de 200 pays

- plus de 1 400 indicateurs

- un historique de plus de 50 ans

Les variables principales sont : country, indicator, year, value.

### 🔍 Pourquoi cette étape est essentielle ?

Avant toute visualisation ou analyse, il est nécessaire de maîtriser parfaitement la base de données.

Cette étape permet :

- d’assurer la cohérence des indicateurs sélectionnés

- de garantir que les séries temporelles sont complètes et fiables

- d’éviter les erreurs dans les corrélations, normalisations ou comparaisons

- de produire un fichier propre, robuste et reproductible

Ce notebook représente donc la fondation technique du projet Beyond GDP,
et garantit la qualité des analyses visibles dans le dashboard final.

In [4]:
import pandas as pd
import os

# 1. Chemins des fichiers

input_path = r"C:\Users\clara\OneDrive\Documents\M2\DataVisualisation\WDICSV.csv"
output_path = r"C:\Users\clara\Downloads\data_dashboard_BeyondGDP.csv"

# Chargement du fichier brut

df = pd.read_csv(input_path, low_memory=False)

# Mise au format long

df = df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    var_name="Year",
    value_name="Value"
)

df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df = df[df["Year"].between(1980, 2023)]

# Sélection d’indicateurs (4 max par catégorie)

variables_selection = {
    # Économie & Productivité
    "NY.GDP.PCAP.CD": "GDP per capita (current US$)",
    "NE.GDI.TOTL.ZS": "Gross capital formation (% of GDP)",
    "FP.CPI.TOTL.ZG": "Inflation, consumer prices (annual %)",

    # Santé & Bien-être
    "SP.DYN.LE00.IN": "Life expectancy at birth (years)",
    "SH.XPD.CHEX.GD.ZS": "Current health expenditure (% of GDP)",
    "SH.DYN.MORT": "Mortality rate, under-5 (per 1,000 live births)",

    # Éducation & Capital humain
    "SE.XPD.TOTL.GD.ZS": "Government expenditure on education (% of GDP)",
    "SE.SEC.ENRR": "School enrollment, secondary (% gross)",
    "HD.HCI.OVRL": "Human capital index (0–1 scale)",

    # Environnement & Énergie
    "EN.GHG.CO2.PC.CE.AR5": "CO₂ emissions per capita (t/person, AR5)",
    "EG.FEC.RNEW.ZS": "Renewable energy consumption (% of total final energy)",
    "EN.ATM.PM25.MC.M3": "PM2.5 air pollution (µg/m³)",

    # Inégalités & Pauvreté
    "SI.POV.GINI": "Gini index",
    "SI.POV.DDAY": "Poverty headcount ratio at $3.65/day (2021 PPP)",
    # "EG.ELC.ACCS.ZS": "Access to electricity (% of population)",

    # Société & Infrastructure
    "SP.URB.TOTL.IN.ZS": "Urban population (% of total population)",
    "SH.H2O.BASW.ZS": "Access to basic drinking water (% of population)"
}

# 5. Filtrage et nettoyage

codes_selection = list(variables_selection.keys())
df_filtered = df[df["Indicator Code"].isin(codes_selection)].copy()

df_filtered["Indicator Name"] = df_filtered["Indicator Code"].map(variables_selection)
df_filtered = df_filtered.drop(columns=["Country Code", "Indicator Code"], errors="ignore")

df_filtered = df_filtered.dropna(subset=["Value"])

# Sauvegarde

os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_filtered.to_csv(output_path, index=False)

print(f"Nouveau fichier sauvegardé : {output_path}")
print("Nombre d'observations :", df_filtered.shape[0])
print("Nombre d'indicateurs uniques :", df_filtered['Indicator Name'].nunique())
print("\nAperçu :")
print(df_filtered.head(10))

Nouveau fichier sauvegardé : C:\Users\clara\Downloads\data_dashboard_BeyondGDP.csv
Nombre d'observations : 114727
Nombre d'indicateurs uniques : 16

Aperçu :
                        Country Name  \
8065296  Africa Eastern and Southern   
8065642  Africa Eastern and Southern   
8065869  Africa Eastern and Southern   
8066421  Africa Eastern and Southern   
8066601  Africa Eastern and Southern   
8066812   Africa Western and Central   
8067158   Africa Western and Central   
8067385   Africa Western and Central   
8067937   Africa Western and Central   
8068117   Africa Western and Central   

                                   Indicator Name  Year       Value  
8065296  CO₂ emissions per capita (t/person, AR5)  1980    1.308881  
8065642              GDP per capita (current US$)  1980  773.439454  
8065869          Life expectancy at birth (years)  1980   49.816713  
8066421    School enrollment, secondary (% gross)  1980   20.370340  
8066601  Urban population (% of total population)  

In [5]:
import pandas as pd

# Charger la base
df = pd.read_csv("data_dashboard_BeyondGDP.csv")

# Nettoyer colonnes
df.columns = [c.strip() for c in df.columns]
df = df.rename(columns={
    "Country Name": "country",
    "Indicator Name": "indicator",
    "Year": "year",
    "Value": "value"
})

# Liste des pays uniques
countries = sorted(df["country"].unique())

print("Nombre de pays :", len(countries))
for c in countries:
    print(c)

Nombre de pays : 265
Afghanistan
Africa Eastern and Southern
Africa Western and Central
Albania
Algeria
American Samoa
Andorra
Angola
Antigua and Barbuda
Arab World
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas, The
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
British Virgin Islands
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Caribbean small states
Cayman Islands
Central African Republic
Central Europe and the Baltics
Chad
Channel Islands
Chile
China
Colombia
Comoros
Congo, Dem. Rep.
Congo, Rep.
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Curacao
Cyprus
Czechia
Denmark
Djibouti
Dominica
Dominican Republic
Early-demographic dividend
East Asia & Pacific
East Asia & Pacific (IDA & IBRD countries)
East Asia & Pacific (excluding high income)
Ecuador
Egypt, Arab Rep.
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Euro area
Europe & Central Asia
Europe & 

In [8]:
# Affiche les 5 premières lignes du dataset
df.head()

,country,indicator,year,value
0,Africa Eastern and Southern,"CO₂ emissions per capita (t/person, AR5)",1980,1.308881
1,Africa Eastern and Southern,GDP per capita (current US$),1980,773.439454
2,Africa Eastern and Southern,Life expectancy at birth (years),1980,49.816713
3,Africa Eastern and Southern,"School enrollment, secondary (% gross)",1980,20.370340
4,Africa Eastern and Southern,Urban population (% of total population),1980,20.900490


In [10]:
# Affiche 10 lignes aléatoires pour voir la variété des données
df.sample(10)

,country,indicator,year,value
11300,Sub-Saharan Africa (excluding high income),Gross capital formation (% of GDP),1987,21.272770
120815,Kuwait,GDP per capita (current US$),2021,34018.634288
53737,Samoa,Urban population (% of total population),2003,21.698000
22073,Malawi,"School enrollment, secondary (% gross)",1992,19.554859
58190,Europe & Central Asia (excluding high income),PM2.5 air pollution (µg/m³),2005,23.172591
35344,IDA blend,Urban population (% of total population),1998,33.633893
113661,Morocco,"School enrollment, secondary (% gross)",2019,79.344051
78064,"Gambia, The",Access to electricity (% of population),2010,46.800000
38161,Sub-Saharan Africa (IDA & IBRD countries),Access to electricity (% of population),1999,30.139507
84869,Austria,"School enrollment, secondary (% gross)",2012,100.108727


In [11]:
# Affiche informations générales : types, colonnes, nombre de valeurs non-null
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128037 entries, 0 to 128036
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   country    128037 non-null  object 
 1   indicator  128037 non-null  object 
 2   year       128037 non-null  int64  
 3   value      128037 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.9+ MB


In [12]:
# Statistiques descriptives (nombre d'observations, min, max, moyenne, etc.)
df.describe(include="all")

,country,indicator,year,value
count,128037,128037,128037.000000,128037.000000
unique,265,18,NaN,NaN
top,United Kingdom,Life expectancy at birth (years),NaN,NaN
freq,625,11650,NaN,NaN
mean,NaN,NaN,2004.897100,947.694218
std,NaN,NaN,11.615415,6331.078667
min,NaN,NaN,1980.000000,-16.859691
25%,NaN,NaN,1996.000000,8.501976
50%,NaN,NaN,2006.000000,37.175000
75%,NaN,NaN,2015.000000,80.936422


In [13]:
# Affiche le nombre total de lignes (observations)
print("Nombre total d'observations :", df.shape[0])

# Affiche le nombre de pays uniques
print("Nombre de pays :", df['country'].nunique())

# Affiche le nombre d'indicateurs dans la base
print("Nombre d'indicateurs :", df['indicator'].nunique())

# Affiche l'intervalle des années couvertes par les données
print("Plage temporelle :", df['year'].min(), "-", df['year'].max())

Nombre total d'observations : 128037
Nombre de pays : 265
Nombre d'indicateurs : 18
Plage temporelle : 1980 - 2023


In [14]:
# Compte le nombre de valeurs manquantes par colonne
df.isna().sum()

country      0
indicator    0
year         0
value        0
dtype: int64

In [25]:
# Compte combien de lignes correspond à chaque indicateur
df['indicator'].value_counts()

indicator
Life expectancy at birth (years)                                        11650
Urban population (% of total population)                                11572
CO₂ emissions per capita (t/person, AR5)                                11044
GDP per capita (current US$)                                            10772
Mortality rate, under-5 (per 1,000 live births)                         10197
Gross capital formation (% of GDP)                                       8515
Renewable energy consumption (% of total final energy)                   8234
Access to electricity (% of population)                                  7859
PM2.5 air pollution (µg/m³)                                              7688
School enrollment, secondary (% gross)                                   6678
Access to basic drinking water (% of population)                         5870
Government expenditure on education (% of GDP)                           5802
Current health expenditure (% of GDP)                 

In [26]:
# Proportion (%) de chaque indicateur
df['indicator'].value_counts(normalize=True).round(3)

indicator
Life expectancy at birth (years)                                        0.091
Urban population (% of total population)                                0.090
CO₂ emissions per capita (t/person, AR5)                                0.086
GDP per capita (current US$)                                            0.084
Mortality rate, under-5 (per 1,000 live births)                         0.080
Gross capital formation (% of GDP)                                      0.067
Renewable energy consumption (% of total final energy)                  0.064
Access to electricity (% of population)                                 0.061
PM2.5 air pollution (µg/m³)                                             0.060
School enrollment, secondary (% gross)                                  0.052
Access to basic drinking water (% of population)                        0.046
Government expenditure on education (% of GDP)                          0.045
Current health expenditure (% of GDP)                 

In [27]:
# Nombre d'années renseignées par pays (du plus complet au moins complet)
df.groupby("country")["year"].nunique().sort_values()

country
Afghanistan                    44
Africa Eastern and Southern    44
Africa Western and Central     44
Albania                        44
Algeria                        44
                               ..
West Bank and Gaza             44
World                          44
Yemen, Rep.                    44
Zambia                         44
Zimbabwe                       44
Name: year, Length: 265, dtype: int64

In [28]:
# Nombre d'indicateurs présents pour chaque pays
df.groupby("country")["indicator"].nunique().sort_values()

country
St. Martin (French part)     4
Channel Islands              4
Isle of Man                  6
Gibraltar                    7
Liechtenstein                8
                            ..
Belgium                     18
Algeria                     18
Zambia                      18
Zimbabwe                    18
Albania                     18
Name: indicator, Length: 265, dtype: int64

In [29]:
# Année min, année max, et nombre d'années renseignées par indicateur
df.groupby("indicator")["year"].agg(["min", "max", "nunique"])

,min,max,nunique
indicator,,,
Access to basic drinking water (% of population),2000,2022,23
Access to clean fuels and technologies for cooking (% of population),2000,2022,23
Access to electricity (% of population),1990,2023,34
"CO₂ emissions per capita (t/person, AR5)",1980,2023,44
Current health expenditure (% of GDP),2000,2023,24
GDP per capita (current US$),1980,2023,44
Gini index,1980,2023,44
Government expenditure on education (% of GDP),1980,2023,44
Gross capital formation (% of GDP),1980,2023,44


In [30]:
# Identifie les pays/indicateurs où toutes les valeurs sont identiques (soucis dans la série)
flat_series = df.groupby(["country", "indicator"])["value"].nunique()
flat_series[flat_series == 1]

country                indicator                                                           
American Samoa         Government expenditure on education (% of GDP)                          1
Andorra                Access to clean fuels and technologies for cooking (% of population)    1
                       Access to electricity (% of population)                                 1
Antigua and Barbuda    Access to clean fuels and technologies for cooking (% of population)    1
Australia              Access to clean fuels and technologies for cooking (% of population)    1
                                                                                              ..
United States          Access to clean fuels and technologies for cooking (% of population)    1
                       Access to electricity (% of population)                                 1
Uruguay                Access to clean fuels and technologies for cooking (% of population)    1
Uzbekistan             Human capita

In [31]:
# Nombre total de lignes dupliquées
df_filtered.duplicated().sum()

np.int64(0)

In [32]:
# Affiche les doublons s'il y en a
df_filtered[df_filtered.duplicated()]

,Country Name,Indicator Name,Year,Value
